In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=dd289edc6faa76edc2a3fc878bdc395d6bd932084c336853f0d81e6635f6a502
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
import pandas as pd
import logging
logging.getLogger().setLevel(logging.DEBUG)

jd = pd.read_csv('data.csv')
# https://github.com/binoydutt/Resume-Job-Description-Matching/blob/master/data.csv
print(jd)



     Unnamed: 0                          company  \
0             1          Visual BI Solutions Inc   
1             2                       Jobvertise   
2             3           Santander Consumer USA   
3             4   Federal Reserve Bank of Dallas   
4             5                           Aviall   
..          ...                              ...   
152         153                     HMS Holdings   
153         154       Kubota Tractor Corporation   
154         155         Deloitte & Touche L.L.P.   
155         156       Kubota Tractor Corporation   
156         157         Deloitte & Touche L.L.P.   

                                              position  \
0    Graduate Intern (Summer 2017) - SAP BI / Big D...   
1                            Digital Marketing Manager   
2      Manager, Pricing Management Information Systems   
3                 Treasury Services Analyst Internship   
4                                Intern, Sales Analyst   
..                         

In [4]:
jd[['position']]

,position
0,Graduate Intern (Summer 2017) - SAP BI / Big D...
1,Digital Marketing Manager
2,"Manager, Pricing Management Information Systems"
3,Treasury Services Analyst Internship
4,"Intern, Sales Analyst"
...,...
152,Business Analyst Intern
153,"Intern, Marketing Analysis -"
154,"Summer Scholar, Business Analyst - Human Capit..."
155,"Intern, Turf Marketing -"


In [5]:


word_counts = jd['Job Description'].str.split().str.len()

# Calculate mean, min, and max
mean_word_count = word_counts.mean()
min_word_count = word_counts.min()
max_word_count = word_counts.max()

print(f'Mean number of words: {mean_word_count}')
print(f'Minimum number of words: {min_word_count}')
print(f'Maximum number of words: {max_word_count}')
print(word_counts.median())


Mean number of words: 524.7070063694267
Minimum number of words: 45
Maximum number of words: 1269
479.0


In [6]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses, models, evaluation
from sentence_transformers.readers import InputExample

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.


In [7]:
data = [InputExample(texts = [{'tag':row['position']},{'doc':row['Job Description']}],label=1.0) for index,row in jd[['position','Job Description']].iterrows()]

In [8]:
# word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')
word_embedding_model = models.Transformer('distilbert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139190078632144 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/150367d8744161cd17b3f6462a14f3a9648752da.lock
DEBUG:filelock:Lock 139190078632144 acquired on /root/.cache/huggingface/hub/.locks/models--dis

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139190078632144 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/150367d8744161cd17b3f6462a14f3a9648752da.lock
DEBUG:filelock:Lock 139190078632144 released on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/150367d8744161cd17b3f6462a14f3a9648752da.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/model.safetensors HTTP/1.1" 302 0
DEBUG:filelock:Attempting to acquire lock 139190068812224 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063.lock
DEBUG:filelock:Lock 139190068812224 acquired on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063.lock
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn-lfs.huggingface.co:443
DEBUG:urllib3.connectionpool:https://cdn-lfs.huggingf

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139190068812224 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063.lock
DEBUG:filelock:Lock 139190068812224 released on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/5e3f1108e3cb34ee048634875d8482665b65ac713291a7e32396fb18f6ff0063.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139190078635312 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/a661b1a138dac6dc5590367402d100765010ffd6.lock
DEBUG:filelock:Lock 139190078635312 acquired on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/a661b1a138dac6dc5590367402d100765010ffd6.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /distilbert-base-uncased/resolve/main/tokenizer_config.json HTTP/1.1" 200 28


...

DEBUG:filelock:Attempting to release lock 139190078635312 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/a661b1a138dac6dc5590367402d100765010ffd6.lock
DEBUG:filelock:Lock 139190078635312 released on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/a661b1a138dac6dc5590367402d100765010ffd6.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139190046334144 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:filelock:Lock 139190046334144 acquired on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /distilbert-base-uncased/resolve/main/vocab.txt HTTP/1.1" 200 231508


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139190046334144 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:filelock:Lock 139190046334144 released on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/fb140275c155a9c7c5a3b3e0e77a9e839594a938.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/tokenizer.json HTTP/1.1" 200 0
DEBUG:filelock:Attempting to acquire lock 139190068821872 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/949a6f013d67eb8a5b4b5b46026217b888021b88.lock
DEBUG:filelock:Lock 139190068821872 acquired on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/949a6f013d67eb8a5b4b5b46026217b888021b88.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /distilbert-base-uncased/resolve/main/tokenizer.json HTTP/1.1" 200 466062


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 139190068821872 on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/949a6f013d67eb8a5b4b5b46026217b888021b88.lock
DEBUG:filelock:Lock 139190068821872 released on /root/.cache/huggingface/hub/.locks/models--distilbert-base-uncased/949a6f013d67eb8a5b4b5b46026217b888021b88.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/added_tokens.json HTTP/1.1" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /distilbert-base-uncased/resolve/main/special_tokens_map.json HTTP/1.1" 404 0


In [9]:
word_embedding_model.get_word_embedding_dimension()

768

In [10]:
doc_model = models.Dense(word_embedding_model.get_word_embedding_dimension(), 768)
# CNN and more layers can be added based on data, and pooling can be moved post CNN
tag_model = models.Dense(word_embedding_model.get_word_embedding_dimension(), 768)

asym_model = models.Asym({
    'tag': [tag_model],
    'doc': [doc_model]
     })
model = SentenceTransformer(modules=[word_embedding_model,pooling_model,asym_model])



INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [11]:


data[0].label = 0.98
train_examples = data
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)


# train_loss = losses.ContrastiveLoss(model=model)
train_loss = losses.CosineSimilarityLoss(model=model)


In [12]:
for i in range(1):
  model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = 30, evaluator=evaluation.EmbeddingSimilarityEvaluator.from_input_examples(data), warmup_steps= 100)

for i in range(1):
  model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = 100, evaluator=evaluation.EmbeddingSimilarityEvaluator.from_input_examples(data), warmup_steps= 100)

Epoch:   0%|          | 0/30 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:EmbeddingSimilarityEvaluator: Evaluating the model on  dataset after epoch 0:
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Cosine-Similarity :	Pearson: -0.1120	Spearman: -0.1201
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Manhattan-Distance:	Pearson: 0.0890	Spearman: 0.1042
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Euclidean-Distance:	Pearson: 0.0854	Spearman: 0.1025
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Dot-Product-Similarity:	Pearson: -0.1372	Spearman: -0.1254


Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:EmbeddingSimilarityEvaluator: Evaluating the model on  dataset after epoch 1:
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Cosine-Similarity :	Pearson: -0.0520	Spearman: -0.0654
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Manhattan-Distance:	Pearson: 0.1139	Spearman: 0.1166
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Euclidean-Distance:	Pearson: 0.1092	Spearman: 0.1166
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Dot-Product-Similarity:	Pearson: -0.1136	Spearman: -0.1254


...

INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:EmbeddingSimilarityEvaluator: Evaluating the model on  dataset after epoch 50:
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Cosine-Similarity :	Pearson: 0.9948	Spearman: 0.1378
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Manhattan-Distance:	Pearson: -0.1059	Spearman: -0.1201
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Euclidean-Distance:	Pearson: -0.0086	Spearman: 0.0071
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Dot-Product-Similarity:	Pearson: 0.6926	Spearman: 0.1378


Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:EmbeddingSimilarityEvaluator: Evaluating the model on  dataset after epoch 51:
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Cosine-Similarity :	Pearson: 0.9951	Spearman: 0.1378
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Manhattan-Distance:	Pearson: -0.0979	Spearman: -0.1166
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Euclidean-Distance:	Pearson: 0.0033	Spearman: 0.0194
INFO:sentence_transformers.evaluation.EmbeddingSimilarityEvaluator:Dot-Product-Similarity:	Pearson: 0.7000	Spearman: 0.1378


Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
position_list = [{'tag':row['position']} for id,row in jd.iterrows()]
position_list

[{'tag': 'Graduate Intern (Summer 2017) - SAP BI / Big Data / Analytics / Visualization'},
 {'tag': 'Digital Marketing Manager'},
 {'tag': 'Manager, Pricing Management Information Systems'},
 {'tag': 'Treasury Services Analyst Internship'},
 {'tag': 'Intern, Sales Analyst'},
 {'tag': 'Human Resources Analyst Internship'},
 {'tag': 'Intern - Business Analytics'},
 {'tag': 'Intern - Business Analytics'},
 {'tag': 'Digital Marketing Intern'},
 {'tag': 'Data Analyst - Intern'},
 {'tag': 'Summer Scholar Analyst - Business Technology, Consulting - Cornell University'},
 {'tag': 'Data Scientist Intern'},
 {'tag': 'Digital Marketing Intern'},
 {'tag': 'Intern (non-IT)'},
 {'tag': 'INTERN, Business Strategy'},
 {'tag': 'Business Risk Services Intern - Summer'},
 {'tag': 'Information Technology Services Intern-Deloitte Application Studios Global Employee Services (GES) Technical - Dallas'},
 {'tag': 'Data Quality Intern (multiple openings)'},
 {'tag': 'Intern, Research & Development'},
 {'tag': 

In [14]:
position_embeddings = model.encode(position_list)


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
position_embeddings.shape

(157, 768)

In [16]:
position_embeddings_norm = np.divide(position_embeddings,np.linalg.norm(position_embeddings,axis=1)[:,np.newaxis])

In [17]:
desc_list = [{'doc':row['Job Description']} for id,row in jd.iterrows()]
# desc_list
desc_embeddings = model.encode(desc_list)
print(desc_embeddings.shape)
desc_embeddings_norm = np.divide(desc_embeddings,np.linalg.norm(desc_embeddings,axis=1)[:,np.newaxis])

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

(157, 768)


In [18]:
# print(desc_embeddings[i])

In [19]:
def topn(i,n=7):
  cosine_sim = np.dot(position_embeddings_norm,desc_embeddings_norm[i])
  sorted_indexes = np.argsort(-cosine_sim)
  return jd.loc[sorted_indexes[0:n],'position']

In [20]:
def similarn(i,n=7):
  cosine_sim = np.dot(position_embeddings_norm,position_embeddings_norm[i])
  sorted_indexes = np.argsort(-cosine_sim)
  return jd.loc[sorted_indexes[0:n],'position']

In [27]:
# predict job title, Testing ...
i = 11
# i = 45
# i = 85
desc = jd[['position','Job Description']].iloc[i,0]
print(" Actual title: ",desc)


# cosine_sim = np.dot(position_embeddings_norm,desc_embeddings_norm[i])
# sorted_indexes = np.argsort(-cosine_sim)
# jd.loc[sorted_index[0:10],'position']
print("Prediction of ith Doc",topn(i))
print('Similar jobs',similarn(i))

 Actual title:  Data Scientist Intern
Prediction of ith Doc 51    Intern - Data Scientist
31    Intern - Data Scientist
62    Intern - Data Scientist
82    Intern - Data Scientist
78     Intern - Data Engineer
47     Intern - Data Engineer
28     Intern - Data Engineer
Name: position, dtype: object
Similar jobs 11       Data Scientist Intern
109       Data Engineer Intern
76     Data Scientist - Intern
45     Data Scientist - Intern
51     Intern - Data Scientist
62     Intern - Data Scientist
31     Intern - Data Scientist
Name: position, dtype: object


In [ ]:
# ROUGH

In [ ]:
# data = [InputExample(texts = [row['position'],row['Job Description']],label=0.86) for index,row in jd[['position','Job Description']].iterrows()]

In [23]:

# from sentence_transformers import SentenceTransformer, InputExample, losses, models, util
# from torch.utils.data import DataLoader

# # Step 1: Create a word embedding model using a pre-trained Transformer model
# word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')

# # Step 2: Create a pooling model to aggregate the word embeddings into a single sentence embedding
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# # Step 3: Create a Sentence Transformer model by passing a list of modules
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# # Step 4: Prepare your dataset in a suitable format for the task and the loss function
# train_examples = [InputExample(texts=['A man is eating food.', 'A man is eating a piece of bread.'], label=0.7346),
#                   InputExample(texts=['A man is eating food.', 'The girl is carrying a baby.'], label=0.2106),
#                   # ... More examples ...
#                   ]

# # data[0].label = 0.98
# # train_examples = data
# # Step 5: Create a dataloader to load and batch your data
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

# # Step 6: Choose a loss function that matches your task and data format
# train_loss = losses.CosineSimilarityLoss(model)
# # train_loss = losses.ContrastiveLoss(model)

# dev_examples = [InputExample(texts=['A plane is taking off.', 'An air plane is flying.'], label=0.875),
#                 InputExample(texts=['A plane is taking off.', 'A bird is flying.'], label=0.229),
#                 # ... More examples ...
#                 ]


# # Step 5.1: Create an evaluator object using the development set
# evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(train_examples, name='sts-dev')

# # Step 7: Train or fine-tune your model using the fit method
# model.fit(train_objectives=[(train_dataloader, train_loss)],
#           epochs=1,
#           evaluator = evaluator,
#           warmup_steps=100
#           )

In [24]:
# asd = train_examples[0]
# print(asd.texts, asd.label, asd.guid)
# asd = data[0]
# # asd.label = 0.89
# print(asd.label, asd.guid)